In [ ]:
import numpy as np
import matplotlib
import scipy.stats as sst
import scipy as sp
import matplotlib.pylab as plt
from ipywidgets import interact,Checkbox,Text,VBox

import scipy.spatial.distance as ssp
import random

In [ ]:
dx = 100
dy = dx

A = np.random.normal(70, 15, size=(50,50))
vector_A = A.flatten()
# print (A)

i_coords, j_coords = np.meshgrid(range(A.shape[0]), range(A.shape[1]), indexing='ij')
coords = np.column_stack((np.concatenate(i_coords), np.concatenate(j_coords)))
index = np.arange(A.shape[0]*A.shape[1])

In [ ]:
unique = random.sample(range(A.shape[0]*A.shape[1]), 150)
data=vector_A[unique]
#Normalization of data
data = (data-np.mean(data))/np.std(data)
len_data = len(data)

In [ ]:
plt.hist(data, density=True)

In [ ]:
plt.hist(data, 
         density=True,
         cumulative=True)

In [ ]:
#CORRECT!!!
coordinates_data = coords[unique]
print(coordinates_data)

In [ ]:
H = ssp.pdist(coordinates_data, metric='euclidean')
D = ssp.squareform(H)

In [ ]:
plt.imshow(D)
plt.colorbar()

In [ ]:
plt.scatter(coordinates_data[:,0], coordinates_data[:,1])
plt.title('Plan view')

In [ ]:
var = np.zeros((len_data,len_data))
for i in range(len_data):
    for j in range(i, len_data):
        var[i,j] = (data[i] - data[j])**2.

In [ ]:
dists_ix = np.triu_indices(D.shape[0], 0)
dists_triu = D[dists_ix]

vars_ix = np.triu_indices(var.shape[0], 0)
vars_triu = var[vars_ix]

plt.scatter(dists_triu, vars_triu, s=40, facecolors='none', edgecolors='b')
plt.xlabel("distance")
plt.ylabel("variance")

In [ ]:
bins = np.array([0.0, 250., 500., 1000., 2000., 3000., 4000., 5000., 6000.])

ixs = []    # for measurement values at point 0
ixs_h = []  # for measurement values at point 1
rs = []     # for rs in each bin
d_mean = [] # for mean distances

for cur_i in range(0, len(bins)-1):
    ix = np.where(np.logical_and(D>bins[cur_i], D<=bins[cur_i+1]))
    d_mean.append(D[ix].mean())
    F_0 = data[ix[0]]
    F_1 = data[ix[1]]
    r = np.corrcoef(F_0, F_1)[1,0]
    rs.append(r)
    ixs.append(F_0)
    ixs_h.append(F_1)  

In [ ]:
plt.plot(d_mean, rs, 'o-')
plt.title('Correlogram')

In [ ]:
Z1, Z2 = np.meshgrid(data, data)
sem_var = (Z1-Z2)**2.


sem_vars = []
co_vars = []
for cur_i_bin, cur_bin in enumerate(bins[:-1]):
    ix = np.where(np.logical_and(D>bins[cur_i_bin], D<=bins[cur_i_bin+1]))
    
    ## do it via the prepared large matrices
    sem_var_cur = np.sum(sem_var[ix])/(2.0 * ix[0].shape[0])
    #print sem_var_cur
    sem_vars.append(sem_var_cur)
    
    mean = data[ix[1]].mean()
    covar = np.sum((data[ix[1]]-mean)* (data[ix[0]]-mean))/( ix[0].shape[0])
    co_vars.append(covar)

In [ ]:
# Plot Empirical Variogram
centers =  (bins[:-1] + bins[1:])/2
plt.scatter(centers, np.array(sem_vars), s=80)
plt.grid(True)
plt.xlabel('distance (h)')
plt.ylabel('$\gamma^{\star}$')
plt.xlim(0.0,)
plt.ylim(0.0,)

In [ ]:
import helpers.theo_variogram_fct as vario
import helpers.th_cov_func as nest_vario
# Define Dictionaries of Theoretical Variogram Models

# Dictionary for individual variogram models. Reference the theo_variogram_fct python file
models = {'Exponential': vario.ExponentialVariogram,
          'Spherical': vario.SphericalVariogram,
          'Gaussian': vario.GaussianVariogram,
          'Linear': vario.LinearVariogram}

# Dictionary for individual short-form variogram models to be used for composited variogram models
mods = {'Exp': vario.ExponentialVariogram,
        'Sph': vario.SphericalVariogram,
        'Gau': vario.GaussianVariogram,
        'Lin': vario.LinearVariogram}

In [ ]:
# plotting for function for individual models
def plot_varmodel(Sill, Range, Model):
    v_model = Model(Sill, Range, h)
    plt.plot(h,v_model, c='red') # Model Variogram
    plt.scatter(centers, np.array(sem_vars), s=80) # Empirical Variogram
    plt.xlabel('distance')
    plt.ylabel('$\\gamma$')
    plt.xlim(0.,max(h))
    plt.ylim(0.,)
    plt.grid(True)
    plt.show()

In [ ]:
# Determine first guess of sill and range from empirical variogram
var_sill = np.round(np.max(np.array(sem_vars)),decimals = -1)
h = np.linspace(0, max(centers),  int(max(centers)/10.))
var_range = np.round(np.max(np.array(centers)),decimals = -1)


In [ ]:
# Trigger Interactive Plot
import importlib
importlib.reload(vario)
interact(plot_varmodel,
         Sill =(var_sill/2,var_sill*2,1),
         Range = (0,max(h), max(h)/100),
         Model = models)

In [ ]:
#Montecarlo random path
index_to_simulate = np.delete(index, unique, axis=0)#Delete from the matrix the 'observed data'
seeds = [10] #Len define number of realizations
for i in seeds:#iteration in every realization 
    random.seed(i)
    path = random.randint(len(index_to_simulate),size=len(index_to_simulate))
    coord_order = coords[index_to_simulate[path]]
    
